# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [ ]:
import pandas as pd
from numpy.random import RandomState
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [ ]:
df_0 = pd.read_csv('sample_data/geo_data_0.csv')
df_1 = pd.read_csv('sample_data/geo_data_1.csv')
df_2 = pd.read_csv('sample_data/geo_data_2.csv')

In [ ]:
def inf(df):
    
    df.info()
    print()
    print(df.head())
    print()
    print(df.describe())
    print()
    print(df['id'].value_counts().head(10))
    print()
    print(df.hist(figsize = (5,5)));

In [ ]:
inf(df_0)

In [ ]:
inf(df_1)

In [ ]:
inf(df_2)

In [ ]:
def delit(df):
    return(df.drop('id', axis = 1))

In [ ]:
df_0 = delit(df_0)

df_1 = delit(df_1)

df_2 = delit(df_2)

# промежуточный вывод:
 - мы открыли данные сохранили в переменные наши датафреймы
 - вывели первую информацию о датафреймах, с нашими даннами все в порядке, нулевых и пустых значений нет, все данные в порядке.
 - вывели графики и посмотрели что данные распределнны нормально.
 - вывели подробную информацию о наших датасетах
 - Данные полностью нормальные можно переходить к следующей задаче
 - Удалили столбец ID, что бы не мешался при обучение данных и прогнозирования модели, а так же там были повторяющиеся ID, видимо появились при человеческом факторе
 
 

## Обучение и проверка модели

In [ ]:
# разделили на признаки и целевой признак каждый датафрейм
def separation(df):
    
    features = df.drop(['product'], axis = 1)
    target = df['product']
    
    return features, target

In [ ]:
features_0, target_0 = separation(df_0)

features_1, target_1 = separation(df_1)

features_2, target_2 = separation(df_2)

In [ ]:
#разделим выборки на тренировочную и валидационную выборку в соотношение 75 на 25
def split(features, target):
    
    features_train, features_valid, target_train, target_valid= train_test_split (
        features, target, test_size = 0.25, random_state = 42
    )
    
    return features_train, features_valid, target_train, target_valid

In [ ]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = split(features_0, target_0)

features_train_1, features_valid_1, target_train_1, target_valid_1 = split(features_1, target_1)

features_train_2, features_valid_2, target_train_2, target_valid_2 = split(features_2, target_2)

In [ ]:
print('features_train_0:',len(features_train_0))
print('target_train_0:', len(target_train_0))
print('features_valid_0:', len(features_valid_0))
print('target_valid_0:',len(target_valid_0))
print()
print('features_train_1:',len(features_train_1))
print('target_train_1:',len(target_train_1))
print('features_valid_1:',len(features_valid_1))
print('target_valid_1:', len(target_valid_1))
print()
print('features_train_2:',len(features_train_2))
print('target_train_2:',len(target_train_2))
print('features_valid_2:',len(features_valid_2))
print('target_valid_2:', len(target_valid_2))

In [ ]:
#Провели масштабирование признаков, привели их к ближайшим значениям
pd.options.mode.chained_assignment = None

def standard_scaler (features_train, features_valid):
    
    num = list(features_train.select_dtypes(exclude ='object').columns)
    
    scaler= StandardScaler()
    scaler.fit(features_train[num])
    
    features_train[num] = scaler.transform(features_train[num])
    features_valid[num] = scaler.transform(features_valid[num])
    
    return features_train , features_valid

data_train = [features_train_0, features_train_1, features_train_2]
data_valid = [features_valid_0, features_valid_1, features_valid_2]

for i in range (len(data_train)):
    standard_scaler(data_train[i], data_valid[i])

In [ ]:
#выведем отмасштабированные признаки
print(len(features_train_0))
print(len(features_valid_0))

In [ ]:
features_train_1.isna().sum()

In [ ]:
#обучили и предсказали модели сохранили RMSE и средний запас сырья в переменные и выевели их
def predictions_model (features_train, target_train, features_valid, target_valid):
    model = LinearRegression()

    model.fit (features_train, target_train)
    predictions_valid = model.predict(features_valid)

    result = mean_squared_error(target_valid, predictions_valid) ** 0.5
    
    return result, predictions_valid.mean(), predictions_valid


In [ ]:
RMSE_0, result_mean_0, predictions_valid_0 = predictions_model(
    features_train_0, target_train_0, features_valid_0, target_valid_0
)

print(f'RMSE: {RMSE_0}, Средний запас сырья: {result_mean_0}')
print()

In [ ]:
RMSE_1, result_mean_1, predictions_valid_1 = predictions_model(
    features_train_1, target_train_1, features_valid_1, target_valid_1
)

print(f'RMSE: {RMSE_1},  Средний запас сырья: {result_mean_1}')
print()

In [ ]:
RMSE_2, result_mean_2, predictions_valid_2 =predictions_model(
    features_train_2, target_train_2, features_valid_2, target_valid_2
)

print(f'RMSE: {RMSE_2}, Средний запас сырья: {result_mean_2}')

 # Вывод:
 - разделили выборки на тренировочную и валидационную
 - отмасштабировали признаки
 - обучили модель и сделали предсказания на валидационной выборке, сохранили RMSE и средний запас сырья в переменные
 - исходя из RMSE можем сделать первоначальный вывод что наша модель лучше всего предсказала данные по второму региону, (Чем ниже RMSE тем модель точнее предсказывает), так же мы видим что средний запас сырья в этом регионе меньше, чем в остальных двух регионах. 

## Подготовка к расчёту прибыли

In [ ]:
BUDGET = 10_000_000_000 # бюджет на разработку скважин
REVENUE_UNIT_PROD = 450_000 # доход с каждой единице продукта
NUMBER_POINTS = 500 # исследуем 500 точек
BEST_POINTS = 200 # по условию выбираем 200 лучших точек

In [ ]:
#Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины
breakeven = BUDGET/ BEST_POINTS/ REVENUE_UNIT_PROD

print(f'Для безубыточной разработки новой скважины: {breakeven}')
print(f'Регион 1: {result_mean_0}, отклонения: {result_mean_0 - breakeven} ')
print(f'Регион 2: {result_mean_1}, отклонения: {result_mean_1 - breakeven}')
print(f'Регион 3: {result_mean_2}, отклонения: {result_mean_2 - breakeven}')

# промежуточный вывод:
- исходя из данных можем увидеть что для каждого из трех регионов разработка 200 скважин будет убыточная. 
- самый убыточны регион 2, минимально убыточный регион 3

## Расчёт прибыли и рисков 

In [ ]:
def income(target_valid, predictions_valid, BEST_POINTS):
    
    target = target_valid.reset_index(drop = True)
    predictions = pd.Series(predictions_valid, index = target.index)
    best_points_target = target[predictions.sort_values(ascending = False).index][:BEST_POINTS]
    
    return (best_points_target.sum() * REVENUE_UNIT_PROD - BUDGET) / 1_000_000_000

income_0 = income(target_valid_0, predictions_valid_0, BEST_POINTS)
income_1 = income(target_valid_1, predictions_valid_1, BEST_POINTS)
income_2 = income(target_valid_2, predictions_valid_2, BEST_POINTS)

print(f'Прибыль для полученного объема сырья, 1 региона: {income_0} млрд.руб.')
print()
print(f'Прибыль для полученного объема сырья, 2 региона: {income_1} млрд.руб.')
print()
print(f'Прибыль для полученного объема сырья, 3 региона: {income_2} млрд.руб.')

# промежуточный вывод:
- минимальная прибыль у 2 региона, самая большая прибыль у 1 региона 

In [ ]:
def region_pred(target_valid, predictions_valid, BEST_POINTS):
    state = RandomState(12345) 
    values = []
    for i in range (1000):
    
        target_subsample = target_valid.reset_index(drop= True).sample(n=NUMBER_POINTS, replace=True, random_state=state)
        probs_subsample = predictions_valid[target_subsample.index]
        
        values.append(income(target_subsample, probs_subsample, BEST_POINTS))
    
    values = pd.Series(values)
    lower = values.quantile(0.025)
    up = values.quantile(0.975)
    interval = values.quantile(0), values.quantile(1)
    mean = values.mean()
    loss_point = (values < 0).sum()
    loss = values[values< 0]
    risk = (values < 0).mean()
    
    print(f'Доверительный интервал: {interval} млрд.руб.')
    print()
    print(f'95,7%-Квантиль: {up} млрд.руб.')
    print()
    print(f'2,5%-Квантиль: {lower} млрд.руб.')
    print()
    print(f'Количество убыточных точек: {loss_point} шт.')
    print()
    print(f'Максимальная прибыль: {values.max()} млрд. руб.')
    print()
    print(f'Средняя прибыль: {mean} млрд.руб.')
    print()
    print(f'Максимальный убыток: {loss.min()} млрд. руб.')
    print()
    print(f'риск убытков: {risk : .2%}')

In [ ]:
print(f'Регион 1')

region_pred(target_valid_0, predictions_valid_0, BEST_POINTS)

In [ ]:
print(f'Регион 2')

region_pred(target_valid_1, predictions_valid_1, BEST_POINTS)

In [ ]:
print(f'Регион 3')

region_pred(target_valid_2, predictions_valid_2, BEST_POINTS)

## Вывод:
- первым нашим шагом было проанализировать датасет и посмотреть на первичные данные и как они распределены:
  - сначала мы проверили все 3 региона на данные
  - удалили столбец 'id' потому, что он нам только мешался и в ходе анализа было установленно что в нем имеют двойные значения, было решено его удалить и сбросить индексы
- вторым нашим шагом было то, что мы обучили линейную регрессию и сделали предсказание по региону краткие выводы по нашей модели:
  - исходя из RMSE можем сделать первоначальный вывод что наша модель лучше всего предсказала данные по второму региону, (Чем ниже RMSE тем модель точнее предсказывает), так же мы видим что средний запас сырья в этом регионе меньше, чем в остальных двух регионах.
- третим шагом мы подготовили данные для расчета:
  - внесли ключевые данные необходимые для расчётов
  - Рассчитали достаточный объём сырья для безубыточной разработки новой скважины, и он равен: 111.11111111111111
- четвертым шагом мы расчитали прибыль для полученного объёма сырья.
  - Прибыль для полученного объема сырья, 1 региона: 3.359141114462179 млрд.руб.

  - Прибыль для полученного объема сырья, 2 региона: 2.4150866966815108 млрд.руб.

  - Прибыль для полученного объема сырья, 3 региона: 2.5985717593741113 млрд.руб.
- пятым нашим шагом посчитали риски и прибыль для каждого региона:
  - Регион 1
    - риск убытков:  6.7%
  - Регион 2
    - риск убытков:  1.90%
  - Регион 3
    - риск убытков:  7.40%
# Рекомендации для бизнеса:
  - Самый оптимальный регион для добычи нефти нашей компании является "Регион 2":
    - 2 регион получил минимальную оценку RMSE =  0.8902801001028845 это говорит о том что наша модель предсказала много правильных ответов, и шанс ошибиться в скважине уменьшается
    - У 2 региона самый минимальный риск убытков = 1.90%
    - средняя прибыль Региона 2 выше чем у других регионов и равняется: 0.37736219242291646 млрд.руб
    - Доверительный интервал (-0.22150203070141983, 1.1654920487038993) млрд.руб. 2 региона, а значит что мы выйдем быстрее в безубыточность, чем в других регионах.